# 🔬 Test de Causalidad de Granger

El Test de Causalidad de Granger (Granger, 1969) es una técnica estadística que se utiliza para determinar si una serie de tiempo, $X$, es útil para predecir otra serie de tiempo, $Y$.

**Es importante notar:** La "causalidad" en este contexto es puramente **predictiva** y **estadística**, no implica necesariamente una causalidad real o estructural en el sentido físico.

## 1. 📝 Fundamento Teórico

### A. La Pregunta Central

Se dice que la variable $X$ **causa Granger** a la variable $Y$ si el historial de $X$ contiene información que ayuda a predecir $Y$ mejor que la información contenida únicamente en el historial de $Y$ (y otras variables relevantes).

### B. Hipótesis del Test

El test se basa en comparar dos modelos: un modelo restringido y un modelo no restringido (Modelo de Vectores Autorregresivos - VAR).

| Modelo | Ecuación |
| :---: | :--- |
| **Restringido ($H_0$):** | $Y_t = \sum_{i=1}^{p} \beta_i Y_{t-i} + \epsilon_t$ |
| **No Restringido ($H_a$):** | $Y_t = \sum_{i=1}^{p} \beta_i Y_{t-i} + \sum_{j=1}^{p} \alpha_j X_{t-j} + u_t$ |

* **Hipótesis Nula ($H_0$):** $X$ **NO** causa Granger a $Y$. (Es decir, todos los coeficientes $\alpha_j$ son cero).
* **Hipótesis Alternativa ($H_a$):** $X$ **SÍ** causa Granger a $Y$. (Al menos un coeficiente $\alpha_j$ es distinto de cero).

### C. Requisito Crucial: Estacionariedad

El Test de Causalidad de Granger requiere que las series de tiempo sean **estacionarias** (media, varianza y autocorrelación constante en el tiempo) o, al menos, **cointegradas**.

* Si las series no son estacionarias, los resultados del test son **espurios** (falsos o sin sentido estadístico).
* **Acción Requerida:** Si las series originales no son estacionarias, primero se deben diferenciar hasta que lo sean (e.g., usar los cambios $\Delta X_t$ y $\Delta Y_t$).

In [1]:
# # 🐍 Implementación del Test de Causalidad de Granger
#
# **Requisitos:** `pandas`, `numpy`, `statsmodels.tsa.stattools`

# %%
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import grangercausalitytests
from statsmodels.tsa.stattools import adfuller
import warnings
warnings.filterwarnings("ignore")

# --- 1. Generación de Series de Tiempo ---
np.random.seed(42)
N = 500  # Número de observaciones

# Crear dos series independientes al principio
ruido_x = np.random.normal(0, 1, N)
ruido_y = np.random.normal(0, 1, N)

# Inicializar las series
X = np.zeros(N)
Y = np.zeros(N)

# Construir X y Y, donde X SÍ causa Granger a Y
# Y_t = 0.5 * Y_{t-1} + 0.3 * X_{t-1} + ruido_y
# X_t = 0.8 * X_{t-1} + ruido_x  (X no depende de Y)
for t in range(1, N):
    X[t] = 0.8 * X[t-1] + ruido_x[t]
    Y[t] = 0.5 * Y[t-1] + 0.3 * X[t-1] + ruido_y[t]

df = pd.DataFrame({'X': X, 'Y': Y})
print(f"✅ Series sintéticas X y Y creadas (N={N}).")

# --- 2. Verificación de Estacionariedad (Importante) ---
# Si las series no son estacionarias, se deben diferenciar.
# El Test ADF (Augmented Dickey-Fuller) verifica la estacionariedad.

def check_stationarity(series, name):
    result = adfuller(series)
    p_value = result[1]
    if p_value < 0.05:
        print(f"   - {name}: p-value={p_value:.4f} < 0.05. Estacionaria.")
    else:
        print(f"   - {name}: p-value={p_value:.4f} >= 0.05. ¡No estacionaria! (Debe diferenciarse)")
    return p_value

print("-" * 50)
print("Verificando Estacionariedad (Test ADF):")
p_x = check_stationarity(df['X'], 'Serie X')
p_y = check_stationarity(df['Y'], 'Serie Y')

# Si una serie no es estacionaria, se usa su primera diferencia (ΔYt)
if p_x >= 0.05 or p_y >= 0.05:
    print("\n⚠️ Al menos una serie no es estacionaria. Usando primeras diferencias para el test Granger.")
    df['dX'] = df['X'].diff().dropna()
    df['dY'] = df['Y'].diff().dropna()
    series_a = df['dY']
    series_b = df['dX']
else:
    series_a = df['Y']
    series_b = df['X']

print("-" * 50)

# --- 3. Aplicar el Test de Causalidad de Granger ---
# Se prueba si series_b (X) causa Granger a series_a (Y).

max_lags = 5  # Número máximo de rezagos (lags) a probar
test_results = grangercausalitytests(df[['Y', 'X']], max_lags, verbose=False)

✅ Series sintéticas X y Y creadas (N=500).
--------------------------------------------------
Verificando Estacionariedad (Test ADF):
   - Serie X: p-value=0.0000 < 0.05. Estacionaria.
   - Serie Y: p-value=0.0000 < 0.05. Estacionaria.
--------------------------------------------------


In [2]:
# --- 4. Interpretación de Resultados ---

# La función prueba la causalidad en ambas direcciones para cada número de rezagos.
# Analizaremos la causalidad de X -> Y y Y -> X en el rezago óptimo (e.g., lag=2)

target_lag = 2 # Foco en un rezago específico para la interpretación
print(f"🔬 Resultados del Test de Causalidad de Granger (Enfoque en Rezago {target_lag}):")

# A. Prueba: ¿X causa Granger a Y? (Y_t = f(Y_{t-i}, X_{t-j}))
p_x_to_y = test_results[target_lag][0]['ssr_ftest'][1]

# B. Prueba: ¿Y causa Granger a X? (X_t = f(X_{t-i}, Y_{t-j}))
# La función grangercausalitytests corre [Y, X] y prueba X -> Y, y [X, Y] y prueba Y -> X.
# La función de statsmodels prueba ambas direcciones, pero requiere reordenar los datos o correrla dos veces.
# Aquí asumimos que la salida para el primer par [Y, X] nos da Y_t = f(Y_t-i, X_t-j)

# Para ser precisos en la dirección opuesta, debemos re-ejecutar:
test_results_reverse = grangercausalitytests(df[['X', 'Y']], target_lag, verbose=False)
p_y_to_x = test_results_reverse[target_lag][0]['ssr_ftest'][1] # X_t = f(X_t-i, Y_t-j)

alpha = 0.05

print("\n--- Dirección: X -> Y (X predice Y) ---")
print(f"   - H0: X no causa Granger a Y. (p-value: {p_x_to_y:.4f})")
if p_x_to_y < alpha:
    print(f"   ▶️ **Conclusión:** Rechazamos H0. X SÍ causa Granger a Y (X es útil para predecir Y).")
else:
    print(f"   ▶️ **Conclusión:** No rechazamos H0. X NO causa Granger a Y.")

print("\n--- Dirección: Y -> X (Y predice X) ---")
print(f"   - H0: Y no causa Granger a X. (p-value: {p_y_to_x:.4f})")
if p_y_to_x < alpha:
    print(f"   ▶️ **Conclusión:** Rechazamos H0. Y SÍ causa Granger a X (Y es útil para predecir X).")
else:
    print(f"   ▶️ **Conclusión:** No rechazamos H0. Y NO causa Granger a X.")

🔬 Resultados del Test de Causalidad de Granger (Enfoque en Rezago 2):

--- Dirección: X -> Y (X predice Y) ---
   - H0: X no causa Granger a Y. (p-value: 0.0000)
   ▶️ **Conclusión:** Rechazamos H0. X SÍ causa Granger a Y (X es útil para predecir Y).

--- Dirección: Y -> X (Y predice X) ---
   - H0: Y no causa Granger a X. (p-value: 0.5287)
   ▶️ **Conclusión:** No rechazamos H0. Y NO causa Granger a X.


### 🚀 Interpretación de los Cuatro Posibles Resultados

#### Después de probar ambas direcciones (X -> Y y Y -> X), se tienen cuatro escenarios:

1.  **Causalidad Unidireccional (X $\to$ Y):** Solo se rechaza $H_0$ en la prueba X $\to$ Y.
2.  **Causalidad Unidireccional (Y $\to$ X):** Solo se rechaza $H_0$ en la prueba Y $\to$ X.
3.  **Causalidad Bidireccional / Feedback:** Se rechaza $H_0$ en **ambas** pruebas (X $\to$ Y y Y $\to$ X). Ambas series se predicen mutuamente.
4.  **No Causalidad:** **No** se rechaza $H_0$ en ninguna de las pruebas. Las series son independientes predictivamente.